In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Nov 21 16:45:19 2019

@author: Khushwant Rai
"""
from sklearn.manifold import TSNE
import numpy as np
import skipthought
from sklearn.cluster import KMeans, SpectralClustering
from sklearn.metrics import pairwise_distances_argmin_min
from keras.layers import Dense, Input, BatchNormalization
from keras.layers import PReLU, Subtract, Concatenate
from keras.models import Model
import scipy
from scipy.sparse.csgraph import laplacian
import matplotlib.pyplot as plt
from keras.regularizers import l1
from collections import Counter
from nltk.tokenize import word_tokenize,sent_tokenize
from rouge import Rouge
from keras.optimizers import Nadam
from nltk.stem.porter import PorterStemmer
import tensorflow_hub as hub
import tensorflow as tf
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
import operator
from tqdm import tqdm
from nltk.corpus import stopwords
import re

class isummary():
    tsne = TSNE(n_components=2, random_state=0, perplexity=30)
    #get the data from dataset
    def get_data(self):
        import pandas as pd
        df=pd.read_csv('cnn_dataset.csv')
        x=df.iloc[:,[1]].values
        y=df.iloc[:,[2]].values
        return x,y            
     #tokenize sentences      
    def split_sentences(self,data):
        sent=[]
        sentences = sent_tokenize(data)
        for j in range(len(sentences)):
            sentences[j] = sentences[j].strip()
            if sentences[j] != '':
                sent.append(sentences[j])
        return sent 
      #sentence mbedding for clustering based model      
    def skipthought_encode(self,data):
        all_sentences = [sent for sent in data]
        print('Loading pre-trained models...')
        model = skipthought.load_model()
        encoder = skipthought.Encoder(model)
        print('Encoding sentences...')
        enc_sentences = encoder.encode(all_sentences, verbose=0)
        print(enc_sentences)
        return enc_sentences
            
    #clustering based refered model
    def summarize(self,data,x): 
        mean = []
        closest = []
        split=self.split_sentences(data)
        print('tokeniztion done')
        vectors = self.skipthought_encode(split)
        print('vetorization done')
        n_clusters = int(x)
        kmeans = KMeans(n_clusters=n_clusters, random_state=0)
        #ts_vec=self.tsne.fit_transform(vectors)
        ts_vec=vectors
        kmeans = kmeans.fit(ts_vec)
        for j in range(n_clusters):
            i = np.where(kmeans.labels_ == j)[0]
            mean.append(np.mean(i))
        closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_,ts_vec)
        ordering = sorted(range(n_clusters), key=lambda k: mean[k])
        summary = ' '.join([split[closest[kr]] for kr in ordering])
        print('summary formed')
        return summary, kmeans, ts_vec, vectors
    
    #autoencoder
    def auto_encoder(self,enc):
        input_img = Input(shape=(4800,))
        encoded = Dense(200, activation='relu', activity_regularizer=l1(10e-6))(input_img)
        encoded = Dense(50, activation='relu')(encoded)
        encoded = Dense(10, activation='linear')(encoded)
        
        decoded = Dense(10, activation='relu')(encoded)
        decoded = Dense(50, activation='relu')(decoded)
        decoded = Dense(200, activation='linear')(decoded)
        decoded = Dense(4800)(decoded)
        autoencoder = Model(input_img, decoded)
    
        encoder = Model(input_img, encoded)
        
        autoencoder.compile(loss=['mse'],
                                loss_weights=[0.99],
                                optimizer=Nadam(0.0002, 0.5))
        
        train_x=enc
        split_size = int(train_x.shape[0]*0.9)
        train_x, val_x = train_x[:split_size], train_x[split_size:]
        
        history=autoencoder.fit(train_x, train_x, epochs=200,
                                batch_size=5, validation_data=(val_x, val_x))
        pred = encoder.predict(train_x)
        training_loss = history.history['loss']
        test_loss = history.history['val_loss']
        epoch_count = range(1, len(training_loss) + 1)
        # Visualize 
        plt.plot(epoch_count, training_loss, 'r--')
        plt.plot(epoch_count, test_loss, 'b-')
        plt.legend(['Training Loss', 'Test Loss'])
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.show();

        return pred
     
    #clustering based model 1
    def summarize_autoenc(self,data,vectors,x):
        mean = []
        closest = []
        split=self.split_sentences(data)
        print('tokeniztion done')
        #vectors = self.skipthought_encode(split)
        print('vetorization done')
        n_clusters = int(x)
        latent=self.auto_encoder(vectors)
        kmeans2 = KMeans(n_clusters=n_clusters, random_state=0)
        ts_vec2=self.tsne.fit_transform(latent)
        kmeans2 = kmeans2.fit(ts_vec2)
        for j in range(n_clusters):
            i = np.where(kmeans2.labels_ == j)[0]
            mean.append(np.mean(i))
        closest, _ = pairwise_distances_argmin_min(kmeans2.cluster_centers_,ts_vec2)
        ordering = sorted(range(n_clusters), key=lambda k: mean[k])
        summary = ' '.join([split[closest[kr]] for kr in ordering])
        print('summary formed')
        return summary, kmeans2, ts_vec2, latent
    
    #clustering based model 2
    def summarize_spectral(self,data,vectors,x, latent):
        mean = []
        closest = []
        split=self.split_sentences(data)
        print('tokeniztion done')

        print('vetorization done')
        n_clusters = int(x)
        spectral = SpectralClustering(affinity='rbf',coef0=1,degree=3,eigen_solver=None,
                            eigen_tol=0.0,gamma=1.0,n_clusters=n_clusters, random_state=0)
        spectral = spectral.fit(latent)
        af=spectral.affinity_matrix_
        lp=laplacian(af,normed=True)
        eig_val, eig_vect = scipy.sparse.linalg.eigs(lp, n_clusters)
        X = eig_vect.real
        rows_norm = np.linalg.norm(X, axis=1, ord=2)
        lp_latent = (X.T / rows_norm).T

        kmeans3 = KMeans(n_clusters=n_clusters, random_state=0)
        ts_vec3 = lp_latent
        kmeans3 = kmeans3.fit(ts_vec3)
        for j in range(n_clusters):
            i = np.where(kmeans3.labels_ == j)[0]
            mean.append(np.mean(i))
        closest, _ = pairwise_distances_argmin_min(kmeans3.cluster_centers_,ts_vec3)
        ordering = sorted(range(n_clusters), key=lambda k: mean[k])
        summary3 = ' '.join([split[closest[kr]] for kr in ordering])
        print('summary formed')
        return summary3, kmeans3, ts_vec3

    #rouge evaluation
    def evaluate_rouge(self, refs, summ):
        rouge = Rouge()
        ref=self.split_sentences(refs)
        out=self.split_sentences(summ)
        kscore={}
        scores = []
        for i in range(len(out)):  
          hypothesis = out[i]
          reference = ref[i]
          scores.append(rouge.get_scores(hyps=hypothesis,refs=reference)[0])

        sums = dict(Counter(scores[0]['rouge-1']) + Counter(scores[1]['rouge-1']))
        mean = {k: v / len(ref) for k, v in sums.items()}
        kscore.update({'rouge-1': mean})
          
        return kscore


    ########frequecy based model############
    
    #prepare frequency table
    def freq(self,text_string):
        stop = set(stopwords.words('english'))
        words = word_tokenize(text_string)
        porter = PorterStemmer()
        
        table = {}
        
        for w in words:
            w = porter.stem(w)
            if w in stop: 
                continue
            elif w in table:
                table[w] += 1
            else:
                table[w] = 1           
        return table
    
    #score sentences 
    def score(self, sent, table):
        sent_val = {}
        for s in sent:
            count = len(word_tokenize(s))
            for val in table:
                if val.lower() in s.lower():                
                    if s in sent_val:
                        sent_val[s] += table[val]
                    else:
                        sent_val[s] = table[val]
            sent_val[s] = sent_val[s] // count
        return sent_val
        
    #summarize
    def freq_summary(self, sent, sent_val,c):
        count = 0
        summary = ''
        for s in sent:
            if s in sent_val and count<c:
                summary += " " + s
                count += 1        
        return summary
    
    ##########graph based model#######
    
    #sentence embedding
    def get_universal_encoder(self,sent):
  
        url = "https://tfhub.dev/google/universal-sentence-encoder/2"
        
        embed = hub.Module(url)
        
        tf.logging.set_verbosity(tf.logging.ERROR)
        with tf.Session() as session:
            session.run([tf.global_variables_initializer(), tf.tables_initializer()])
            vectors = session.run(embed(sent))
        return vectors
    
  ###############adversarial autoencoder#################
  
      #encoder and decoder setup
    def encoder(self, latent_dim, cat_dim, input_dim):
        inputvec = Input(shape=(4800,))
        encoded = Dense(200, activation='relu')(inputvec)
        encoded = Dense(50, activation='relu')(encoded)
        category = Dense(10)(encoded)
        category = BatchNormalization()(category)
        category = PReLU()(category)
        category = Dense(cat_dim, activation='softmax')(category)
        
        latent = Dense(50)(encoded)
        latent = BatchNormalization()(latent)
        latent = PReLU()(latent)
        latent = Dense(latent_dim, activation='linear')(latent)
    
        decoded = Concatenate()([latent, category])
        decoded = Dense(50, activation='relu')(decoded)
        decoded = Dense(200, activation='relu')(decoded)
        decoded = Dense(4800)(decoded)
        error = Subtract()([inputvec, decoded]) 
        model = Model(inputvec, [decoded, latent, category, error])
        return model
    
    #generates random vector from normal distribution
    def random_normal(self, latent_dim, batch_size, window_size=None):
        shape = (batch_size, latent_dim) 
        return np.random.normal(size=shape)
    
    #generate random one hot encoded vectors
    def random_categories(self, categ, batch_size):
        cats = np.zeros((batch_size, categ))
        for i in range(batch_size):
            one = np.random.randint(0, categ)
            cats[i][one] = 1
        return cats
     
    #discriminator setup
    def discriminator(self, latent_dim):
        input_layer = Input(shape=(latent_dim,))
        disc = Dense(128)(input_layer)
        disc = Dense(64)(disc)
        disc = Dense(1, activation="sigmoid")(disc)
        model = Model(input_layer, disc)
        return model
    
    #training discriminator
    def train_disrim(self, discriminator, real, fake):
        def train(real_samples, fake_samples):
            discriminator.trainable = True
    
            loss_real = discriminator.train_on_batch(real_samples, real)
            loss_fake = discriminator.train_on_batch(fake_samples, fake)
            loss = np.add(loss_real, loss_fake) * 0.5
    
            discriminator.trainable = False
    
            return loss
        return train
    
    #summarizing adversarial encoder generated summary 
    def adversarial_summary(self,pdata, length,vect):
        split=self.split_sentences(pdata)
        mean = []
        closest = []
        kmeans4 = KMeans(n_clusters=length, random_state=0)
        kmeans4 = kmeans4.fit(vect)
        for j in range(length):
            i = np.where(kmeans4.labels_ == j)[0]
            mean.append(np.mean(i))
        closest, _ = pairwise_distances_argmin_min(kmeans4.cluster_centers_,vect)
        ordering = sorted(range(length), key=lambda k: mean[k])
        summary4 = ' '.join([split[closest[kr]] for kr in ordering])  
        return summary4


    
def main():
    m1_rouge=[]
    m2_rouge=[]
    m3_rouge=[]
    m4_rouge=[]
    freq_rouge=[]
    graph_rouge=[]
    obj=isummary()
    x,y=obj.get_data()

    for i in range(100):
        sent = sent_tokenize(x[i][0])
        length=len(sent_tokenize(y[i][0]))
        ######freq based######   
        sent = [re.sub('\n', '', i) for i in sent]
        table = obj.freq(" ".join(sent))
        scores = obj.score(sent, table)
        scores = dict(sorted(scores.items(), key=operator.itemgetter(1)))
        freq_summary = obj.freq_summary(sent, scores,length)
        freq_rouge.append(obj.evaluate_rouge(y[i][0],freq_summary))
        
        ########graph absed#######
        uni_vector=obj.get_universal_encoder(sent)
        cos = cosine_similarity(uni_vector)
        graph = nx.from_numpy_array(cos)
        graph_scores = nx.pagerank(graph)
        ranks = sorted(((graph_scores[i],s) for i,s in enumerate(sent)), reverse=True)
        graph_summary = " ".join([i[1] for i in ranks[:length]])
        graph_rouge.append(obj.evaluate_rouge(y[i][0],graph_summary))
        
        ########cluster based########
        summary,tsvec,kmeans,vector=obj.summarize(x[i][0],length)
        summary2,kmeans2,tsvec2, latent=obj.summarize_autoenc(x[i][0],vector,length)
        summary3,kmeans3,tsvec3=obj.summarize_spectral(x[i][0],vector,length,latent)
        m1_rouge.append(obj.evaluate_rouge(y[i][0],summary))
        m2_rouge.append(obj.evaluate_rouge(y[i][0],summary2))
        m3_rouge.append(obj.evaluate_rouge(y[i][0],summary3))
        
        ############adversarial autoencoder#######
        train_x=vector
        input_dim = train_x.shape[1]
        latent_dim = 10
        cat_dim = length
        discrim_norm = obj.discriminator(latent_dim)
        discrim_norm.compile(loss='binary_crossentropy', 
                                    optimizer='Nadam', 
                                    metrics=['accuracy'])
    
        discrim_norm.trainable = False
    
        discrim_category = obj.discriminator(cat_dim)
        discrim_category.compile(loss='binary_crossentropy', 
                                  optimizer='Nadam', 
                                  metrics=['accuracy'])
    
        discrim_category.trainable = False
    
        encoder = obj.encoder(latent_dim, cat_dim, input_dim)
    
        inputvec = Input(shape=(4800,))
        reconstructed_signal, encoded_repr, category, _ = encoder(inputvec)
    
        norm = discrim_norm(encoded_repr)
        categ = discrim_category(category)
    
        autoencoder = Model(inputvec, [reconstructed_signal, norm, categ])
        autoencoder.compile(loss=['mse', 'binary_crossentropy', 'binary_crossentropy'],
                                        optimizer='Nadam')
    
        batches = 2000
        batch_size=64
    
        lenc = []
        lval = []
        rl = np.ones((batch_size, 1))
        fk = np.zeros((batch_size, 1))
    
        train_discrim_norm = obj.train_disrim(discrim_norm, rl, fk)
        train_dicrim_categ = obj.train_disrim(discrim_category, rl, fk)
    
        start = tqdm(range(batches))
    
        for _ in start:
          
            ids = np.random.randint(0, train_x.shape[0], batch_size)
            vect = train_x[ids]
    
            _, flatent, fcategory, _ = encoder.predict(vect)
    
            rlatent = obj.random_normal(latent_dim, batch_size)
            rcategory = obj.random_categories(cat_dim, batch_size)
    
            norm_loss = train_discrim_norm(rlatent, flatent)
            categ_loss = train_dicrim_categ(rcategory, fcategory)
            
            enc_loss = autoencoder.train_on_batch(vect, [vect, rl, rl])
            lenc.append(enc_loss)
    
            val_loss = autoencoder.test_on_batch(vect, [vect, rl, rl])
            lval.append(val_loss)
    
            start.set_description("[normal_disc_acc:  %.2f%% Category_disc_acc: %.2f%%] [MSE train: %f val: %f]"
                                 %(100*norm_loss[1], 100*categ_loss[1], enc_loss[1], val_loss[1]))
            
        (dec, rep, cat, error) = encoder.predict(train_x)
        summary4=obj.adversarial_summary(x[i][0],length,rep)
        m4_rouge.append(obj.evaluate_rouge(y[i][0],summary4))
        plt.plot([loss[1] for loss in lenc])
        plt.plot([loss[1] for loss in lval])
        plt.legend(['Training Loss', 'Test Loss'])

if __name__ == '__main__':
    main()